# 读写文本格式的数据
anaconda ipython shell  
pandas Input/Output pd.read_csv()  
pandas Input/Output pd.read_table()  
python Built-in Functions open()  
python Built-in Functions list()  
pandas General functions pd.isna()

In [ ]:
import pandas as pd
import numpy as np
!cat ch06/ex1.csv

In [ ]:
df=pd.read_csv('ch06/ex1.csv',sep=',')
df

In [ ]:
pd.read_table('ch06/ex1.csv',sep=',')

In [ ]:
!cat ch06/ex2.csv

In [ ]:
pd.read_csv('ch06/ex2.csv',header=None)

In [ ]:
pd.read_csv('ch06/ex2.csv',names=['a','b','c','d','message'])

In [ ]:
names = ['a','b','c','d','message']
pd.read_csv('ch06/ex2.csv',names=names,index_col='message')

In [ ]:
!cat ch06/csv_mindex.csv

In [ ]:
parsed= pd.read_csv('ch06/csv_mindex.csv',index_col=['key1','key2'])
parsed

In [ ]:
list(open('ch06/ex3.txt'))

In [ ]:
result=pd.read_table('ch06/ex3.txt',sep='\s+')
result

In [ ]:
!cat ch06/ex4.csv

In [ ]:
pd.read_csv('ch06/ex4.csv',skiprows=[0,2,3])

In [ ]:
!cat ch06/ex5.csv

In [ ]:
result=pd.read_csv('ch06/ex5.csv')
result

In [ ]:
pd.isna(result)

In [ ]:
result=pd.read_csv('ch06/ex5.csv',na_values=['NULL'])
result

In [ ]:
sentinels={'message':['foo','NA'],'something':['two']}
pd.read_csv('ch06/ex5.csv',na_values=sentinels)

## 逐块读取文本文件
pandas Input/Output TextFileReader  
pandas series s.value_counts()  
pandas series s.add()  
pandas series s.sort_values()

In [ ]:
result = pd.read_csv('ch06/ex6.csv')
result.head()

In [ ]:
pd.read_csv('ch06/ex6.csv',nrows=5)

In [ ]:
chunker=pd.read_csv('ch06/ex6.csv',chunksize=1000)
tot=pd.Series([])
for piece in chunker:
    tot =tot.add(piece.loc[:,'key'].value_counts(),fill_value=0)
tot = tot.sort_values(ascending=False)
tot.iloc[:10]

## 将数据写出到文本格式
pandas dataframe d.to_csv()  
python sys s.stdout  
numpy Array creation routines np.arange()  
pandas General functions pd.date_range()  
pandas Index pd.DatetimeIndex  
pandas series s.to_csv()

In [ ]:
data = pd.read_csv('ch06/ex5.csv')
data

In [ ]:
data.to_csv('ch06/out.csv')
!cat ch06/out.csv

In [ ]:
import sys
data.to_csv(sys.stdout,sep='|')

In [ ]:
data.to_csv(sys.stdout,na_rep='NULL')

In [ ]:
data.to_csv(sys.stdout,index=False,columns=['a','b','c'])

In [ ]:
dates= pd.date_range('1/1/2000',periods=7)
ts = pd.Series(np.arange(7),index=dates)
ts.to_csv('ch06/tseries.csv')
!cat ch06/tseries.csv

In [ ]:
#实现pd.Series.from_csv()功能
print(pd.Series.from_csv('ch06/tseries.csv',parse_dates=True))
pd.read_csv('ch06/tseries.csv',parse_dates=True,index_col=0,header=None,names=['value']).iloc[:,0]

## 手工处理分隔符格式
python csv csv.reader()  
python Built-in Functions zip()  
python csv Dialect.delimiter  
python csv csvreader.__next__()  
python csv csv.writer()  
python csv csvwriter.writerow()

In [ ]:
!cat ch06/ex7.csv

In [ ]:
import csv
f = open('ch06/ex7.csv')
reader = csv.reader(f)
for line in reader:
    print(line)

In [ ]:
lines=list(csv.reader(open('ch06/ex7.csv')))
header,values = lines[0],lines[1:]
data_dict = {h: v for h, v in zip(header,zip(*values))}
data_dict

In [ ]:
#reader = csv.reader(f,delimiter='|')
with open('ch06/mydata.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(('one','two','three'))
    writer.writerow(('1','2','3'))
!cat ch06/mydata.csv

## JSON数据
python json json.loads()  
python json json.dumps()  
python dict d[key]  
pandas dataframe d.to_json()  
pandas Input/Output pd.read_json()

In [ ]:
obj = """
{"name": "Wes",
"places_lived": ["United States","Spain","Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age":25, "pet": "Zuko"},
            {"name":"Katie", "age":33,"pet":"Cisco"}]
}
"""
import json
result = json.loads(obj)
result #字典对象

In [ ]:
asjson=json.dumps(result)
asjson

In [ ]:
siblings=pd.DataFrame(result['siblings'],columns=['name','age'])
siblings

In [ ]:
result_df =siblings.to_json()
siblings2 =pd.read_json(result_df)
print(result_df)
siblings2

## XML和HTML: Web信息收集
python urllib.request urlopen()  
lxml lxml.html Functions parse()  
lxml \_ElementTree e.getroot()  
lxml HtmlElement etree.\_Element  
lxml \_Element e.findall()  
lxml \_Element e.get()  
lxml HtmlMixin h.text_content()

In [ ]:
from lxml.html import parse
from urllib.request import urlopen
parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc =parsed.getroot()
links=doc.findall('.//a')
links[15:20]

In [ ]:
lnk =links[0]
lnk

In [ ]:
lnk.get('href')

In [ ]:
lnk.text_content()

In [ ]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[:10]

In [ ]:
tables =doc.findall('.//table')
calls =tables[0]
puts =tables[1]
rows =calls.findall('.//tr')

def _unpack(row,kind='td'):
    """
    抽象数据行的提取方法
    lxml _Element e.findall()
    python List Comprehensions 列表推导
    python Text Sequence Type — str s.format()
    """   
    elts =row.findall('.//{0}'.format(kind))
    return [val.text_content() for val in elts]

print(_unpack(rows[0],kind='th'))
print(_unpack(rows[1],kind='td'))

In [ ]:
def parse_options_data(table):
    """
    抽象表格toDataFrame的提取方法
    pandas dataframe pd.DataFrame 主动使用
    """
    rows = table.findall('.//tr')
    header = _unpack(rows[0],kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return pd.DataFrame(data,columns=header)
call_data =parse_options_data(calls)
put_data =parse_options_data(puts)
call_data.head()

## 利用lxml.objectify解析XML
lxml objectify Functions parse()  
python Built-in Functions open()  
lxml \_ElementTree e.getroot()  
lxml ObjectifiedElement o.getchildren()  
lxml \_Element tag  
lxml NumberElement FloatElement  
lxml NumberElement IntElement  
lxml NumberElement pyval  
lxml StringElement pyval  
python Sequence Types — list, tuple, range s.append()  
python io — Core tools for working with streams io.StringIO  
lxml \_Element e.get()  
lxml \_Element text

In [ ]:
from lxml import objectify
path ='ch06/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root =parsed.getroot()
data=[]
skip_fields=['PARENT_SEQ','INDICATOR_SEQ','DESIRED_CHANGE','DECIMAL_PLACES']
for elt in root.INDICATOR:
    el_data={}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
        data.append(el_data)
perf = pd.DataFrame(data)
perf.head()

In [ ]:
from io import StringIO
tag='<a href="http://www.baidu.com">Baidu</a>'
root = objectify.parse(StringIO(tag)).getroot()
root

In [ ]:
root.get('href')

In [ ]:
root.text